In [1]:
import urllib.request as req
import os.path
import json
import requests
import os
from datetime import datetime, timedelta
import time
import pytz
from dateutil import parser
import pandas as pd
API_ROOT = "https://api.synopticdata.com/v2/"
API_TOKEN = "622863d185934c71a58c12a6e679c23d"
api_arguments = {"token":API_TOKEN}
NWS_TOKEN = "WkXIizXZDpEHxorFvkahwHGgBoDfFxRR"
SMU_API_TOKEN = 'cff7722463364a7ab147d188e0945c25'
import pickle
scaler = pickle.load(open('scaler.sav' , 'rb'))
model = pickle.load(open('Kmeans.sav', 'rb'))

In [ ]:
t_df = pd.DataFrame()
i = -1
while True:
    
    now = time.strftime('%Y%m%d%H%M', time.localtime())
    url = 'https://api.synopticdata.com/v2/stations/precip?stid={}&recent={}&precip=1&units=precip|in&hfmetar=1&obtimezone=local&token={}'.format('KDAL',15,SMU_API_TOKEN)
    req = requests.get(url)
    res = json.loads(req.text)
    if res['SUMMARY']['RESPONSE_CODE'] == 2 : 
        print('no observation, rainless')
    if res['SUMMARY']['RESPONSE_CODE'] == 1 :
        print('True info')
        index = res['STATION'][0]['OBSERVATIONS']['ob_end_time_1']
        #print(index)
        dt = pd.DataFrame(res['STATION'][0]['OBSERVATIONS'] , index=[i])
        dt['query_time'] = now
        #print(now)
        t_df = t_df.append(dt)
        t_df['cumsum'] = t_df['total_precip_value_1'].cumsum()
        t_df['max_int'] = t_df['cumsum'].diff().fillna(0)

        intensity = max(t_df['max_int'])
        print('intensity = ',intensity)
        total_depth = t_df.iloc[len(t_df)-1]['cumsum']
        print('total_depth', total_depth)
        t_df = t_df.set_index(t_df['ob_end_time_1'].apply(
            lambda x: parser.parse(x.split('T')[0].replace('-' , '/') + ' '+x.split('T')[1].split('-')[0])))
        #print(t_df)
        if total_depth > 0:
            X = scaler.transform([[intensity , total_depth]])
            #print(X)
            y = model.predict(X)
            print('storm cluster is: ', y)

            if y == 2:
                print('severe storm')   
            if y == 1:
                print('moderate_storm')
            if y == 0:
                print('light rain')
        if total_depth == 0:
            if t_df['total_precip_value_1'].rolling('3h').max()[len(t_df)-1] == 0:
                print('here rain stoped', )
                y = 'Rainless'
                t_df = pd.DataFrame()
                print(y)
  
        else:
            y = 'Rainless'
            i=-1
            t_df = pd.DataFrame()
            print(y)
            
        print()
    time.sleep(15*60)
 

no observation, rainless
True info
intensity =  0.0
total_depth 0.0
here rain stoped
Rainless

no observation, rainless
True info
intensity =  0.0
total_depth 0.0
here rain stoped
Rainless

no observation, rainless
no observation, rainless
no observation, rainless
True info
intensity =  0.0
total_depth 0.0
here rain stoped
Rainless

no observation, rainless
no observation, rainless
no observation, rainless
True info
intensity =  0.0
total_depth 0.0
here rain stoped
Rainless

no observation, rainless
